### **Inisialisasi**

Seperti fine tuning model lainnya kita harus menentukan model backbone yang akan digunakan

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from trl import SFTTrainer

# Nama model yang ingin di-fine-tune
model_name = "mistralai/Mistral-7B-v0.1"

# Load model dan tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

### **LoRA**

#### **Rank (r)**
Rank dari matriks dekomposisi A dan B. Ini adalah parameter paling krusial dalam LoRA karena secara langsung mengontrol jumlah parameter yang dapat dilatih. Kegunaannya adalah menentukan "kapasitas" atau kompleksitas dari adaptasi yang dapat dipelajari oleh model.
Cara kerjanya:
- Nilai r kecil (misal, 4, 8, 16): Menghasilkan matriks A dan B yang sangat kecil. Jumlah parameter yang dilatih sedikit, proses fine-tuning lebih cepat, dan kebutuhan memori lebih rendah. Ini cocok untuk adaptasi tugas yang tidak terlalu kompleks.
- Nilai r besar (misal, 64, 128, 256): Menghasilkan matriks A dan B yang lebih besar. Model memiliki kapasitas lebih untuk mempelajari adaptasi yang kompleks, namun dengan biaya komputasi dan memori yang lebih tinggi.

#### **LoRA Alpha**

Kegunaan: Mengontrol seberapa besar "magnitudo" atau pengaruh dari matriks adaptasi LoRA terhadap output asli dari lapisan pre-trained. Ini mirip dengan learning rate, tetapi berlaku untuk seluruh matriks adaptasi.
Bagaimana cara kerjanya:
Ketika lora_alpha diatur sama dengan r (misalnya lora_alpha=16 dan r=16), penskalaannya menjadi 1, yang berarti tidak ada penskalaan tambahan.
Mengatur lora_alpha lebih tinggi dari r (misalnya lora_alpha=32 dan r=16) akan memperkuat pengaruh dari adaptasi LoRA.

#### **LoRA Dropout**
Ini adalah dropout rate yang diterapkan pada matriks adaptasi LoRA (biasanya pada matriks B).
Kegunaannya adalah sebagai teknik regulasi untuk mencegah overfitting pada parameter-parameter LoRA yang sedang dilatih.
Cara kerjanya: Selama proses pelatihan, sebagian neuron (unit) dari matriks adaptasi akan dinonaktifkan secara acak pada setiap iterasi. Ini memaksa model untuk tidak terlalu bergantung pada beberapa parameter adaptasi saja, sehingga meningkatkan kemampuannya untuk generalisasi.

In [ ]:
# Konfigurasi LoRA
lora_config = LoraConfig(
    r=8,  # Rank dari matriks adaptasi rendah
    lora_alpha=32, # Skala untuk matriks adaptasi
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # Jenis task
)

In [ ]:
# Persiapkan model untuk pelatihan 8-bit dan tambahkan adapter LoRA
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)


In [ ]:

# Definisikan argumen pelatihan
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    output_dir="./lora_fine_tuned_model",
    fp16=True,
    push_to_hub=False # Set ke True jika ingin menyimpan ke Hugging Face Hub
)

# Definisikan trainer SFT (Supervised Fine-Tuning)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data, # Ganti dengan dataset Anda
    args=training_arguments,
    tokenizer=tokenizer,
    peft_config=lora_config,
    dataset_text_field="text" # Sesuaikan dengan nama kolom teks di dataset Anda
)

# Latih model
trainer.train()

In [ ]:
# Simpan adapter LoRA
trainer.save_model("./lora_adapters")

### **QLoRA dengan Unsloth**

QLoRA atau Quantized Low Rank Adaptation adalah teknik PEFT yang mirip dengan LoRA namun kita fine tuning dengan kuantisasi model LLM ke byte yang lebih kecil seperti 16 bit, 8 bit hingga int 8.

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # quantized to 4-bit, uses less memory
    load_in_8bit = False,    # quantized to 8-bit, uses less memory
    full_finetuning = False, # full finetuning?
    # token = "hf_...",     
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

In [ ]:
from datasets import load_dataset

reasoning_dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [ ]:
def generate_conversation(examples):
    problems  = examples["problem"]
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

In [ ]:
import pandas as pd
chat_percentage = 0.25
non_reasoning_subset = pd.Series(non_reasoning_conversations)
non_reasoning_subset = non_reasoning_subset.sample(
    int(len(reasoning_conversations)*(chat_percentage/(1 - chat_percentage))),
    random_state = 2407,
)
print(len(reasoning_conversations))
print(len(non_reasoning_subset))
print(len(non_reasoning_subset) / (len(non_reasoning_subset) + len(reasoning_conversations)))

In [ ]:
data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(non_reasoning_subset)
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

trainer_stats = trainer.train()